In [2]:
import pint.models as model
import pint.toa as toa
import pint.logging
import pint.fitter
from pint.modelutils import model_ecliptic_to_equatorial
import numpy as np
import astropy.units as u
import pint.derived_quantities as dq
import table_utils as tu
from scipy.special import fdtrc

pint.logging.setup(level="ERROR")

1

In [ ]:
#aplpy download fits cutouts from panstarrs use 1 or 3 to make images with real WCS & tickmarks

**J1816+4510 eclipsing calculations**

For J1816+4510, we see additional delays around superior conjuntion at 820 MHz, probably due to additional electron content around the companion that the signal must travel through when the pulsar eclipses. First I want to figure out the DM bump corresponding to this delay (roughly 800 microseconds).

In [9]:
def SC_dDM(dt,freq):
    """ Calculate additional column density at superior conjunction (given additional timing delay)
    
    Parameters
    ==========
    dt: quantity, additional dispersive timing delay due to plasma around companion
    freq: quantity, observing frequency of measured delay
    
    Returns
    =======
    dDM_sup: quantity, additional column density at superior conjunction (cm^-2)
    """
    Dconst = 4.148808e3*(1e6*u.Hz)**2*u.cm**3*u.s*u.pc**-1 # HBOPA, eqn. 4.6
    dDM_sup = (dt*freq**2/Dconst).to(u.pc*u.cm**-3) # extra DM corresponding to delay
    return dDM_sup.to(u.cm**-2)

To compare to a similar calculation in Freire (2005; ASPCS, p. 407), I convert DM (column density) to 1/cm^2. The result is comparable to the column density of J2051-0827 (Stappers et al. 2001).

In [10]:
dDM_1816 = SC_dDM(800.0*1e-6*u.s,820.0*(1e6*u.Hz))
print(dDM_1816)
print(dDM_1816.to(u.pc*u.cm**-3))

4.0007821153349005e+17 1 / cm2
0.12965651820956764 pc / cm3


How big might the companion be based on the duration of the eclipse? Assume inclination angle of 90 deg for now, and consider an eclipse that lasts some fraction of an orbit (based on gap between ingress-egress).

In [11]:
def psr_comp_separation(model,inc=90.0*u.deg,mp=1.4*u.solMass):
    """ Calculate separation between pulsar and binary companion
    
    Parameters
    ==========
    model: object, PINT timing model
    inc: quantity, assumed orbital inclination (optional, default = 90 deg)
    mp: quantity, assumed pulsar mass (optional, default = 1.4 Msun)
    """
    mc = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=inc, mp=mp)
    return (mo['A1'].quantity/np.sin(inc)).to(u.R_sun)*(1+(mp/mc))

def Rplasma(a, frac_eclipse):
    """ Estimate companion (plasma?) radius based on extent of eclipse
    
    Parameters
    ==========
    a: quantity, separation between the pulsar and companion
    frac_eclipse: float, fraction of the orbit where pulsar signal is obscured
    
    Returns
    =======
    Rplasma: quantity, companion's plasma(?) radius (Solar Radii)
    """
    print(f"Nominal pulsar-companion separation: {a.value:.4f} Rsun")
    ing2sup = frac_eclipse*180.0*u.deg # angle from ingress to superior conjunction
    Rplasma = a*np.sin(ing2sup)
    print(f"Companion's estimated plasma radius: {Rplasma.value:.4f} Rsun ({Rplasma.to(u.km).value:.0f} km)")
    return Rplasma

frac_eclipse = 0.04

par_path = f"data/J1816+4510_eclipseDMX.par"
tim_path = f"data/J1816+4510_eclipseDMX.tim"
mo = model.get_model(par_path)
to = toa.get_TOAs(tim_path,model=mo)
fo = pint.fitter.WLSFitter(to,mo)
x = fo.fit_toas()
a = psr_comp_separation(mo, mp=1.64*u.solMass) # using lower limit on M_psr from Clark et al. (2023)
print(f"J1816+4510 is obscured for {frac_eclipse:.0%} of an orbit...")
rp = Rplasma(a,frac_eclipse)

J1816+4510 is obscured for 4% of an orbit...
Nominal pulsar-companion separation: 2.6041 Rsun
Companion's estimated plasma radius: 0.3264 Rsun (227061 km)


I can use the estimated companion's plasma radius to get some idea of the electron density near the companion (using the additional column density around superior conjunction). This is probably ~order of magnitude.

In [12]:
(dDM_1816/rp).to(u.cm**-3)

<Quantity 17619863.47492632 1 / cm3>

Next, I want to use the equation from Eggleton et al. (1983) to estimate the size of the companion's Roche Lobe, and thus, the path length the pulsar might be encountering around superior conjunction. Do this for a few possible inclination angles and assume pulsar mass is 1.64 solar masses.

In [13]:
def RL_Eggleton(a,mc,mp=1.4*u.solMass):
    """ Calculate Roche Lobe size in solar radii (see Eggleton 1983)
    
    Parameters
    ==========
    a: quantity, separation between the pulsar and companion
    mc: quantity, companion mass (solar masses)
    mp: pulsar mass (default: 1.4 Msun), optional
    
    Returns
    =======
    R_L: quantity, Roche Lobe size (solar radii)
    """
    q = mc/mp
    R_L = 0.49*a*q**(2.0/3)/(0.6*q**(2.0/3)+np.log(1.0+q**(1.0/3)))
    return R_L.to(u.R_sun)

incs = [90.0,85.0,80.0,75.0] # trial inclination angles, degrees

mo = model.get_model(par_path)
to = toa.get_TOAs(tim_path,model=mo)
fo = pint.fitter.WLSFitter(to,mo)
x = fo.fit_toas()

for inc in incs:
    mp = 1.64*u.M_sun
    a = psr_comp_separation(mo,inc=inc*u.deg,mp=mp)
    mc = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=inc*u.deg, mp=mp)
    R_L = RL_Eggleton(a,mc,mp=mp)
    RlRp_ratio = R_L/rp

    mc_str = f"{mc.value:.4f}"
    q_str = f"{mc/mp:.4f}"
    a_str = f"{a.value:.4f}"
    rl_str = f"{R_L.value:.5f}"
    rlrp_str = f"{RlRp_ratio:.2f}"

    print(f"  i = {inc} deg; mc = {mc_str} Msun; q = {q_str}; a = {a_str} Rsun; RL = {rl_str} Rsun; RL/Rp = {rlrp_str}")

  i = 90.0 deg; mc = 0.1792 Msun; q = 0.1093; a = 2.6041 Rsun; RL = 0.55252 Rsun; RL/Rp = 1.69
  i = 85.0 deg; mc = 0.1800 Msun; q = 0.1097; a = 2.6044 Rsun; RL = 0.55324 Rsun; RL/Rp = 1.70
  i = 80.0 deg; mc = 0.1822 Msun; q = 0.1111; a = 2.6055 Rsun; RL = 0.55544 Rsun; RL/Rp = 1.70
  i = 75.0 deg; mc = 0.1860 Msun; q = 0.1134; a = 2.6073 Rsun; RL = 0.55917 Rsun; RL/Rp = 1.71


Therefore (unlike the systems considered in Freire 2005), the Roche Lobe (R_L above) is somewhat larger than the size of the plasma cloud (R_p), so matter responsible for increased dispersive delays is bound to the companion.

**Optical Constraints Calculations**

I follow a procedure similar to that described in Section 4.7 of Swiggum et al. (2023) / Section 7 of Lynch et al. (2018).

In [8]:
def MagLim_to_Lum(Mlim):
    """ Convert magnitude limit to luminosity
    
    Parameter
    =========
    Mlim (float): limiting magnitude(s)
    
    Returns
    =======
    Llim (quantity): luminosity limit(s), solar luminosity
    """
    Msun = 4.74
    Llim=u.solLum*10**((Msun-Mlim)/2.5)
    return Llim.decompose()

def TeffLim(Rlim,Llim):
    """ Limit effective temp based on radius, luminosity
    
    Parameters
    ==========
    Rlim (quantity): estimate of stellar radius, solar radii
    Llim (quantity): estimate of stellar luminosity, solar luminosity
    
    Returns
    =======
    Tlim (quantity): upper limit on Teff, Kelvin
    """
    from astropy.units.cds import c
    sbc = 5.6704e-8*u.W*u.meter**-2*u.K**-4
    Tlim=(Llim/(4*np.pi*Rlim**2*sbc))**0.25
    return Tlim.decompose()

In [130]:
# reddening in PS1 bands from Schlafly & Finkbeiner 2011, Table 6
# grizy
ps1_redden=np.array([3.172,2.271,1.682,1.322,1.087]) # y not constraining?

# assume no detection.  Typical PS1 grizy stack limits are:
ps1_lims = np.array([23.3,23.2,23.1,22.3,21.4])

# extinction from http://argonaut.skymaps.info (see Green et al. 2015)
# these take distance (largest NE vs. YMW) into account
psr = "J0032+6946"
extinction = 0.91

par_path = f"data/{psr}_fiore+23.par"
mo = model.get_model(par_path)

mp = 1.4*u.Msun

pb = (1./mo['FB0'].quantity).to(u.day)

mcmax = dq.companion_mass(pb, mo['A1'].quantity, i=26.0*u.deg, mp=mp)
mcmed = dq.companion_mass(pb, mo['A1'].quantity, i=60.0*u.deg, mp=mp)
mcmin = dq.companion_mass(pb, mo['A1'].quantity, i=90.0*u.deg, mp=mp)

gcoord = mo.coords_as_GAL()
dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,mo['DM'].value)

dmdist = max([dmdist_ne,dmdist_ymw])

distance_mod = 5*np.log10((dmdist*u.kpc/(10.0*u.pc)).decompose())

print(f"{psr}: {mcmin.value:.3f}-{mcmed.value:.3f}-{mcmax.value:.3f} Msun")
grizy_lims = ps1_lims-extinction*ps1_redden-distance_mod # de-reddened limits
print(grizy_lims)

J0032+6946: 0.417-0.496-1.212 Msun
[8.15744868 8.87735868 9.31334868 8.84094868 8.15479868]


In [135]:
import matplotlib.pyplot as plt
from astropy.units import cds

# > 0.3 Msun, WD behaves nicely (cold degenerate matter), has mass-radius relationship
# type of atmosphere can change, H (DA) vs. He (DB)
# abs. magnitude depends on surface temperature, which depends on age & mass (cooling models)
# with PanSTARRS nondetection, limit M -> limit Teff -> limit age for a given mass

m = 0.4956
psr = "J0032+6946"

m, psr= ms[0], psrs[0]

# read in Bergeron tables (https://www.astro.umontreal.ca/~bergeron/CoolingModels/)

def limit(maglims,atmo="DA",m="0.4"):
    check_rows = np.loadtxt(f'data/berg{m}.txt',dtype=str,usecols=2)
    dbrow = np.max(np.where(check_rows=="Mo"))
    if atmo=="DA":
        teffs, g, r, i, z, y, ages = np.loadtxt(f'data/berg{m}.txt',dtype=str,skiprows=2,max_rows=dbrow-2, 
                                                usecols=[0,29,30,31,32,33,42],unpack=True)
        parenthesis = "(DA / H atmosphere)"
    elif atmo=="DB":
        teffs, g, r, i, z, y, ages = np.loadtxt(f'data/berg{m}.txt',dtype=str,skiprows=dbrow+3, 
                                                usecols=[0,29,30,31,32,33,42],unpack=True)
        parenthesis = "(DB / He atmosphere)"
    else:
        print("Atmosphere must be 'DA' or 'DB'")
        return
    grizydata = [g,r,i,z,y]
    bands = ['g','r','i','z','y']
    teff_lims = []
    age_lims = []
    for band, maglim, bergmags in zip(bands, maglims, grizydata):
        idx = (np.abs(maglim - np.array(bergmags).astype(float))).argmin()
        teff, age = teffs[idx], ages[idx]
        teff_lims.append(float(teff))
        age_lims.append(float(age))
        #print(f"{psr}: using values for {m} Msun, {band} band, cooling age {age} yr: {teff} K")
    idx = np.argmin(np.array(teff_lims))
    print(f"{psr}: Mc = {m} Msun, {bands[idx]}-band limited (M_{bands[idx]} < {maglims[idx]:.1f}), cooling age {age_lims[idx]*10**-6:.2f} Myr: {teff_lims[idx]:.0f} K {parenthesis}")

In [136]:
for mass in [0.4,0.5,0.6]:
    limit(maglims=grizy_lims,atmo="DA",m=str(mass))
    limit(maglims=grizy_lims,atmo="DB",m=str(mass))

J0032+6946: Mc = 0.4 Msun, r-band limited (M_r < 8.9), cooling age 0.94 Myr: 35000 K (DA / H atmosphere)
J0032+6946: Mc = 0.4 Msun, i-band limited (M_i < 9.3), cooling age 0.44 Myr: 40000 K (DB / He atmosphere)
J0032+6946: Mc = 0.5 Msun, r-band limited (M_r < 8.9), cooling age 1.74 Myr: 45000 K (DA / H atmosphere)
J0032+6946: Mc = 0.5 Msun, r-band limited (M_r < 8.9), cooling age 1.70 Myr: 47500 K (DB / He atmosphere)
J0032+6946: Mc = 0.6 Msun, r-band limited (M_r < 8.9), cooling age 1.51 Myr: 55000 K (DA / H atmosphere)
J0032+6946: Mc = 0.6 Msun, i-band limited (M_i < 9.3), cooling age 1.40 Myr: 60000 K (DB / He atmosphere)


F-tests for parameter inclusion in timing solutions.

Subscripts "1" are without the parameter in question, "2" includes the parameter.

In [3]:
def ftest(chi2_1, chi2_2, dof_1, dof_2):
    delta_chi2 = chi2_1 - chi2_2
    delta_dof = dof_1 - dof_2
    redchi2_2 = chi2_2 / dof_2
    F = float((delta_chi2 / delta_dof) / redchi2_2) # F-statistic
    ft = fdtrc(delta_dof, dof_2, F) # probability of null hypothesis (?)
    return ft

F-test for PSR J0636+5128 PX.

In [4]:
chi2_1 = 1836.87
chi2_2 = 1834.81
dof_1 = 1403-50
dof_2 = 1403-51
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.21814667466085338


We want this to be below ~0.0027 or so. PX is not significant for J0636.

F-test for PSR J1239+3239 PMDEC

In [139]:
chi2_1 = 346.7
chi2_2 = 339.07
dof_1 = 283-12
dof_2 = 283-13
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.01432787774388313


We want this to be below ~0.0027 or so. PMDEC is not significant for J1239.

F-test for PSR J1239+3239 PBDOT

In [138]:
chi2_1 = 346.7
chi2_2 = 341.23
dof_1 = 283-12
dof_2 = 283-13
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.03842955847403444


We want this to be below ~0.0027 or so. PBDOT is not significant for J1239.

F-test for PSR J1327+3423 PX

In [6]:
chi2_1 = 2131.91
chi2_2 = 2116.83
dof_1 = 1575-47
dof_2 = 1575-48
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.000995443012847512


This is < 0.0027. PX is significant for PSR J1327+3423.

F-test for PSR J1816+4510 F2 (with DMX)

In [10]:
chi2_1 = 1137.24
chi2_2 = 1131.08
dof_1 = 749-81
dof_2 = 749-82
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.057089071613551


This is > 0.0027. With DMX, F2 is not significant for PSR J1816+4510.

F-test for PSR J1816+4510 FB1 (with DMX)

In [11]:
chi2_1 = 1137.24
chi2_2 = 1132.1
dof_1 = 749-81
dof_2 = 749-82
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.08228305805880497


This is > 0.0027. With DMX, FB1 is not significant for PSR J1816+4510.

F-test for PSR J1816+4510 XDOT (with DMX)

In [3]:
chi2_1 = 1444.69
chi2_2 = 1431.13
dof_1 = 749-59
dof_2 = 749-60
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.010830907269635929


This is > 0.0027. With DMX, XDOT is not significant for PSR J1816+4510.

F-test for PSR J0214+5222 XDOT (PINT, third-order ELL1 model)

In [4]:
chi2_1 = 1042.053
chi2_2 = 1028.988
dof_1 = 936
dof_2 = 935
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.0005953038887263694


This is < 0.0027. XDOT is significant for PSR J0214+5222.

F-test for PSR J0214+5222 XDOT (TEMPO2, BT model)

In [6]:
chi2_1 = 1041.96
chi2_2 = 1028.92
dof_1 = 936
dof_2 = 935
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.0006022668980345058


This is < 0.0027. XDOT is significant for PSR J0214+5222.

F-test for PSR J0032+6946 PBDOT (PINT, third-order ELL1 model)

In [3]:
chi2_1 = 1656.660
chi2_2 = 1653.852
dof_1 = 1419
dof_2 = 1418
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.12097377461662188


This is > 0.0027. PBDOT is not significant for PSR J0032+6946.

F-test for PSR J0214+5222 XDOT (PINT, third-order ELL1 model)

In [4]:
chi2_1 = 1042.053
chi2_2 = 1028.988
dof_1 = 936
dof_2 = 935
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.0005953038887263694


This is < 0.0027. XDOT is significant for PSR J0214+5222.

F-test for PSR J0214+5222 PMDEC (PINT, third-order ELL1 model)

In [5]:
chi2_1 = 934.568
chi2_2 = 933.419
dof_1 = 935
dof_2 = 934
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.2838857475473009


This is > 0.0027. PMDEC is not significant for PSR J0214+5222.

F-test for PSR 1816+4510 FB1 (without DMX)

In [140]:
chi2_1 = 2047.67
chi2_2 = 2047.20
dof_1 = 734
dof_2 = 733
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.6817617064484285


This is > 0.0027. FB1 is not significant for PSR J1816+4510.

F-test for PSR 1816+4510 XDOT (without DMX)

In [142]:
chi2_1 = 2047.67
chi2_2 = 1980.53
dof_1 = 734
dof_2 = 733
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

7.745261572086393e-07


This is < 0.0027. XDOT is significant for PSR J1816+4510 in the absence of a DMX fit.

F-test for PSR 1816+4510 EPS1DOT (without DMX)

In [143]:
chi2_1 = 2047.67
chi2_2 = 1961.901995.86
dof_1 = 734
dof_2 = 733
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

2.1622015830690272e-08


This is < 0.0027. EPS1DOT is significant for PSR J1816+4510 in the absence of a DMX fit.

F-test for PSR 1816+4510 EPS2DOT (without DMX)

In [144]:
chi2_1 = 2047.67
chi2_2 = 1995.86
dof_1 = 734
dof_2 = 733
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

1.4729906017427073e-05


This is < 0.0027. EPS2DOT is significant for PSR J1816+4510 in the absence of a DMX fit.

F-tests for testing PSR J1816+4510 binary params (without DMX)

In [146]:
chi2_removeXDOT = 1935.96
chi2_removeEPS1DOT = 1893.09
chi2_removeEPS2DOT = 1938.49
chi2_initial = 1883.11
dof_1 = 732
dof_2 = 731
ft_removeXDOT = ftest(chi2_removeXDOT, chi2_initial, dof_1, dof_2)
ft_removeEPS1DOT = ftest(chi2_removeEPS1DOT, chi2_initial, dof_1, dof_2)
ft_removeEPS2DOT = ftest(chi2_removeEPS2DOT, chi2_initial, dof_1, dof_2)
print(f"Removing XDOT: {ft_removeXDOT}")
print(f"Removing EPS1DOT: {ft_removeEPS1DOT}")
print(f"Removing EPS2DOT: {ft_removeEPS2DOT}")

Removing XDOT: 6.905276346396521e-06
Removing EPS1DOT: 0.04941373908072355
Removing EPS2DOT: 4.195625771467472e-06


F-test for PSR 1816+4510 F2 (without DMX; adding to XDOT, EPS2DOT)

In [147]:
chi2_1 = 1893.09
chi2_2 = 1812.27
dof_1 = 732
dof_2 = 731
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

1.6467761709440166e-08


This is < 0.0027. F2 is significant for PSR J1816+4510 in the absence of a DMX fit.

F-test for PSR 1816+4510 FB1 (without DMX; adding to XDOT, EPS2DOT)

In [150]:
chi2_1 = 1809.39
chi2_2 = 1890.77
dof_1 = 733
dof_2 = 732
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.000887697044361157


This is < 0.0027. FB1 is significant for PSR J1816+4510 in the absence of a DMX fit, but only if F2 is present.

F-test for PSR 1816+4510 FB1 (without DMX; adding to F2, XDOT, EPS2DOT)

In [150]:
chi2_1 = 1812.27
chi2_2 = 1785.03
dof_1 = 731
dof_2 = 730
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.000887697044361157


This is < 0.0027. FB1 is significant for PSR J1816+4510 in the absence of a DMX fit, but only if F2 is present.

F-test for PSR 1816+4510 XDOT (with DMX)

In [148]:
chi2_1 = 1444.41
chi2_2 = 1430.87 # EPS2DOT is 1435.19, FB1 is 1444.39, F2 is 1439.08
dof_1 = 690
dof_2 = 689
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.010881956520331311


This is > 0.0027. XDOT is not significant for PSR J1816+4510 when fitting for DMX.

F-test for PSR 1816+4510 XDOT & EPS2DOT (with DMX)

In [149]:
chi2_1 = 1444.41
chi2_2 = 1422.27
dof_1 = 690
dof_2 = 688
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

0.004923769231877012


This is > 0.0027. XDOT + EPS2DOT is not significant for PSR J1816+4510 when fitting for DMX.

F-test for PSR 1816+4510 DMX

In [4]:
chi2_1 = 1829.15
chi2_2 = 1444.41
dof_1 = 733
dof_2 = 690
ft = ftest(chi2_1, chi2_2, dof_1, dof_2)
print(ft)

1.2505937414492888e-16


This is << 0.0027. DMX is very significant for PSR J1816+4510.